# Text Mining K-Means

In [1]:
#***************************************************************************************
# 1. Cargar el corpus de textos
#***************************************************************************************

print("\n\n1. Cargamos el corpus")
trainCorpus = ["Me gustan las vacas",
               "Me gustan los caballos",
               "odio los perros",
               "odio los caballos",
               "me gustan las ranas",
               "me gusta el helado",
               "no quiero comer",
               "los helados son cremosos"]
print (trainCorpus)



1. Cargamos el corpus
['Me gustan las vacas', 'Me gustan los caballos', 'odio los perros', 'odio los caballos', 'me gustan las ranas', 'me gusta el helado', 'no quiero comer', 'los helados son cremosos']


In [2]:
#*********************************************************************************************************
# 2. Vectorizar los textos del corpus (convertir cada texto en un vector de frecuencias de palabras)
#*********************************************************************************************************
print("\n\n2. Vectorizamos los textos")

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

#Transformar los documentos en una matriz de tf's de documentos.
vectorizer.fit(trainCorpus) #El vectorizador aprende el vocabulario del corpus
print ("Atributos:",vectorizer.get_feature_names_out())
tfMatrix = vectorizer.transform(trainCorpus) #Extraer las frecuencias de palabras (tf)
print ("Matriz tf:\n",tfMatrix.toarray())

#La matriz tf es nuestro dataset, donde:
#  - cada fila representa una muestra (un documento del corpus)
#  - cada columna representa un atributo (la frecuencia de una palabra en dicho documento)
print("nº de muestras: %d, nº de atributos: %d" % tfMatrix.shape)



2. Vectorizamos los textos
Atributos: ['caballos' 'comer' 'cremosos' 'el' 'gusta' 'gustan' 'helado' 'helados'
 'las' 'los' 'me' 'no' 'odio' 'perros' 'quiero' 'ranas' 'son' 'vacas']
Matriz tf:
 [[0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1]
 [1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0]
 [0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0]]
nº de muestras: 8, nº de atributos: 18


In [3]:
#***************************************************************************************
# 3. Clusterizar los documentos mediante el algoritmo K-means
#***************************************************************************************
print("\n\n3. Clusterizamos los textos")

from sklearn.cluster import KMeans
from sklearn import metrics

# Asignar a kmeans un valor de k=2, es decir que el algoritmo intentará encontrar 2 clusters
k=2
km = KMeans(n_clusters=k,max_iter=100)
km.fit(tfMatrix)
print ("Clusters:",km.labels_)
for i in range(k):
    print ("\nCluster",i,":")
    for j in range(km.labels_.size):
        if km.labels_[j]==i:
            print ("\t",trainCorpus[j])



3. Clusterizamos los textos
Clusters: [0 0 1 1 0 0 0 1]

Cluster 0 :
	 Me gustan las vacas
	 Me gustan los caballos
	 me gustan las ranas
	 me gusta el helado
	 no quiero comer

Cluster 1 :
	 odio los perros
	 odio los caballos
	 los helados son cremosos


In [4]:
#***************************************************************************************
# 4. Medir la calidad del cluster
#***************************************************************************************
print("\n\n4. Medimos la calidad de nuestro cluster")

#Ground truth son las categorias correctas puestas a mano (1= texto positivo, 0=negativo), para comparar con las automaticas de kmeans
groundTruth = [1,1,0,0,1,1,0,1] 
print ("Clusters:    ",km.labels_)
print ("Ground Truth:",groundTruth)

# Un cluster es homogéneo si todos sus elementos contienen miembros de una misma clase
print("Homogeneity: %0.3f" % metrics.homogeneity_score(groundTruth, km.labels_))

# Una clase es completa si todos sus elementos pertenecen al mismo cluster
print("Completeness: %0.3f" % metrics.completeness_score(groundTruth, km.labels_))

print("V-measure: %0.3f" % metrics.v_measure_score(groundTruth, km.labels_))



4. Medimos la calidad de nuestro cluster
Clusters:     [0 0 1 1 0 0 0 1]
Ground Truth: [1, 1, 0, 0, 1, 1, 0, 1]
Homogeneity: 0.166
Completeness: 0.166
V-measure: 0.166


In [5]:
#***************************************************************************************
# 5. Usar los clusters previos para clasificar un nuevo texto entrante
#***************************************************************************************
print("\n\n4. Clasificamos un nuevo texto entrante")

testCorpus = ["odio los animales"]
tfMatrixTest =  vectorizer.transform(testCorpus)
print ("Nuevo texto:",testCorpus)
print ("Nueva matriz tf:",tfMatrixTest.toarray() )

prediction = km.predict(tfMatrixTest)[0]
print ("Prediccion: Cluster",prediction)



4. Clasificamos un nuevo texto entrante
Nuevo texto: ['odio los animales']
Nueva matriz tf: [[0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0]]
Prediccion: Cluster 1


##### Analisis del resultado:
Se observa que no lo ha hecho del todo mal: el clúster 0 contiene textos negativos y
el clúster 1 positivos. Parece que ha fallado solo con “los helados son cremosos”,
pues, aunque es positivo, se ha metido en el clúster de los negativos. 
Además,después del entrenamiento es capaz de clasificar un nuevo texto de entrada dentro del clúster correcto: “odio los animales” lo ha clasificado dentro del clúster 0, que era el que contenía textos negativos.